In [44]:
# %pip install pandas

In [ ]:
import altair as alt
import pandas as pd
import json
import os
from IPython.display import display

alt.data_transformers.disable_max_rows()

data = []
for f in os.listdir("data"):
    if f.endswith(".json"):
        new_data = json.load(open(os.path.join("data", f)))
        data.extend(new_data)

flattened_data = [item for sublist in data for item in sublist]
df = pd.DataFrame(flattened_data)
df['duration_ns'] = df['duration'].apply(lambda d: d['secs'] * 1_000_000_000 + d['nanos'])
df = df.drop(columns=['duration'])
# df = df[df['strategy'] != 'MemZero'].copy()


# Define the parameters for grouping the plots
plot_grouping_params = ['total_size']
charts_to_combine = []

plot_grouping_params = ['total_size']
charts_to_combine = []


print("--- Rendering Non-Overlapping Charts ---")
for params, group_df in df.groupby(plot_grouping_params):
    size, resident = params
    
    size_kb = size // 1024
    resident_kb = resident // 1024
    title = f"Durations for Size={size_kb}KB, Resident={resident_kb}KB"

    chart = alt.Chart(group_df).mark_boxplot(extent=1.5, outliers=False).encode(
        x=alt.X('duration_ns:Q', title='Duration (nanoseconds)'),
        y=alt.Y('strategy:N', title='Strategy'),
        
        # KEY CHANGE: Add yOffset to separate the boxplots vertically
        yOffset='dirty_fraction:N',

        # Color still distinguishes the categories
        color=alt.Color('dirty_fraction:N', title='Dirty Fraction')
    ).properties(
        title=title,
        width=700,
        # Increased height to give the dodged plots more space
        height=300 
    )
    
    display(chart)
    charts_to_combine.append(chart)

# --- Combine and Save to Single HTML File ---
if charts_to_combine:
    combined_chart = alt.vconcat(*charts_to_combine)
    combined_chart.save('pagemap-scan-benchmarks.html')
    print("\n--- HTML Generation ---")
    print("Successfully combined all charts into a single file: pagemap-scan-benchmarks.html")

--- Rendering Non-Overlapping Charts ---


KeyError: 'keep_resident'